In [ ]:
# Import libraries

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action="ignore", category=Warning)

****Print the available files****

In [ ]:
files = [file for file in os.listdir("/kaggle/input/sales-2019/")]
for file in files:
    print(file)

****Merge all the files in to single CSV file****

In [ ]:
# Create an empty data frame
all_data = pd.DataFrame()

# Iterate over the files to concatenate
for file in files:
    data = pd.read_csv("/kaggle/input/sales-2019/" + file)
    all_data = pd.concat([all_data, data])

all_data.head()

In [ ]:
all_data.shape

In [ ]:
all_data.info()

In [ ]:
all_data.isnull().sum()

In [ ]:
# drop nulls where all the rows or columns are null
all_data = all_data.dropna(how = "all")

In [ ]:
all_data.isnull().sum()

In [ ]:
all_data.shape

In [ ]:
# check for unique values
all_data["Quantity Ordered"].unique()

In [ ]:
# Filter out the text data from the rest
filter = all_data["Quantity Ordered"] == "Quantity Ordered"
all_data = all_data[~filter]

all_data["Quantity Ordered"].unique()

In [ ]:
all_data.shape

In [ ]:
all_data.dtypes

In [ ]:
# change the data type of the columns 
all_data["Quantity Ordered"] = all_data["Quantity Ordered"].astype(int)
all_data["Price Each"] = all_data["Price Each"].astype(float)
all_data["Order Date"] = pd.to_datetime(all_data["Order Date"])

In [ ]:
all_data.dtypes

In [ ]:
# Create the Sales column for the product
all_data["Sales"] = all_data["Quantity Ordered"] * all_data["Price Each"]
all_data.head()

In [ ]:
# create the City column from the Purchase Address column
def city(x):
    return x.split(",")[1]
all_data["City"] = all_data["Purchase Address"].apply(city)
all_data.head()

In [ ]:
# create Month, Day, Hour column from the Order Date
all_data["Month"] = all_data["Order Date"].dt.month
all_data["Day"]   = all_data["Order Date"].dt.dayofweek
all_data["Hour"]  = all_data["Order Date"].dt.hour
all_data.head()

**Analyzing monthly sales**

In [ ]:
# Find the Sales of each month
all_data.groupby("Month")["Sales"].sum().sort_values(ascending = False).round(1)

In [ ]:
# Visualize using bar plot
monthly_sales = all_data.groupby("Month").sum()

plt.figure(figsize = (10,6))

sns.barplot( y = monthly_sales["Sales"],
             x = monthly_sales.index,
             data = monthly_sales)
plt.title("Total sales by month", fontsize = 15)
plt.ylabel("Sales in USD", fontsize = 12)
plt.xlabel("Months", fontsize = 12)

* Question: what was the best month for sales?
* Answer:   December

In [ ]:
# Find the percentage increment in sales from january to december
all_data.groupby("Month")["Sales"].sum().pct_change(periods = 11)

* What is the percentage change in sales from january to december?
* 153.17 %

**Analyze total sale by hour**

In [ ]:
all_data.groupby("Hour")["Sales"].sum()

In [ ]:
# Visualize using line plot
hourly_sales = all_data.groupby("Hour").sum()

plt.figure(figsize = (10, 6))

sns.lineplot( x = hourly_sales.index,
               y = hourly_sales["Sales"],
               data = hourly_sales)

plt.title("Total Sales by Hour", fontsize = 15)
plt.xlabel("Hour", fontsize = 12)
plt.ylabel("Sales in USD", fontsize = 12)
plt.xticks(ticks = hourly_sales.index)
plt.grid(True)

* Question: What are the peak hours that should be looked at to maximize sales through adevertisement, customer satisfaction, etc?
* Answer: Hours between 9 to 12 and between 16 to 19

**Daily order trend**

In [ ]:
all_data.groupby("Day")["Sales"].sum()

In [ ]:
# visualize using line plot

daily_sales = all_data.groupby("Day").sum()
daily_sales.index = ["Monday", "Tuesday", "Wednesday", " Thursday", "Friday", "Saturday", "Sunday"]

plt.figure(figsize = (10, 6))

sns.lineplot(x = daily_sales.index,
             y = daily_sales["Sales"],
             data = daily_sales)

plt.title("Total Sales by Day", fontsize = 15)
plt.xlabel("Day of week", fontsize = 12)
plt.ylabel("Sales in USD", fontsize = 12)
plt.xticks(ticks = daily_sales.index)
plt.grid()

* Question: Which day of the week has highest and lowest sales?
* Answer: Tuesday has the heighest sales and lowest on Thursday.

**Analyze maximum order by city**

In [ ]:
all_data.groupby("City")["Sales"].sum().sort_values(ascending = False)

In [ ]:
# visualize using bar plot
city_sales = all_data.groupby("City").sum()

plt.figure(figsize = (10, 6))

sns.barplot( x = city_sales["Sales"],
             y = city_sales.index,
             data = city_sales,
             order = city_sales.sort_values("Sales").index)

plt.title("Total Sales by City", fontsize = 15)
plt.xlabel("Sales in USD", fontsize = 12)
plt.ylabel("City", fontsize = 12)


* Question: Which city has the highest and lowest Sales?
* Answer: highest sales with San Francisco and lowest sales with Austin

In [ ]:
# Find the quantity ordered for each product
all_data.groupby("Product")["Quantity Ordered"].sum().sort_values(ascending = False)

In [ ]:
all_data["Product"].str.contains("Google Phone").sum()

In [ ]:
# Find number of times the product is ordered
all_data.groupby("Product")["Quantity Ordered"].count().sort_values(ascending = False)

In [ ]:
# Visualize using barplot
quantity_ordered = all_data.groupby("Product").sum().sort_values( by = "Quantity Ordered", ascending = False)

plt.figure(figsize = (10,6))

sns.barplot( x = quantity_ordered["Quantity Ordered"],
             y = quantity_ordered.index,
             data = quantity_ordered)

plt.title("Quantity Ordered for each Product", fontsize = 15)
plt.xlabel("Quantity Ordered", fontsize = 12)
plt.ylabel("Products", fontsize = 12)

In [ ]:
# Visualize using barplot
product_sales = all_data.groupby("Product").sum().sort_values( by = "Sales", ascending = False)

plt.figure(figsize = (10,6))

sns.barplot( x = product_sales["Sales"],
             y = product_sales.index,
             data = product_sales)

plt.title("product_sales for each Product", fontsize = 15)
plt.xlabel("product_sales in USD", fontsize = 12)
plt.ylabel("Products", fontsize = 12)

* The product with highest qunatity ordered need not have highest sales. Infact AAA Batteries are highest in quantity ordered but least in overall sales

**What products are most often sold together?**

In [ ]:
# Find the duplicate of order_id

df = all_data[all_data["Order ID"].duplicated(keep = False)]
df

In [ ]:
# Make a new column that join the products with same Order ID

df["sold_together"] = df.groupby("Order ID")["Product"].transform(lambda x: ",".join(x))
df

In [ ]:
# Drop duplicates
df = df[["Order ID", "sold_together"]].drop_duplicates()
df

In [ ]:
# Find the top 10 products that are sold together
df.groupby("sold_together")["Order ID"].count().sort_values(ascending = False)[:10]

In [ ]:
# Visualize in bar plot
Sold_together = df.groupby("sold_together").count().sort_values( by = "Order ID", ascending = False)[:10]

plt.figure(figsize = (10,6))

sns.barplot( x = Sold_together["Order ID"],
             y = Sold_together.index,
             data = Sold_together)

plt.title("Top 10 products sold together", fontsize = 15)
plt.xlabel("Total Orders", fontsize = 12)
plt.ylabel("Products_sold_together", fontsize = 12)

* Question: Top products that are sold together?
* Answer: iphone, Lightning Charging Cable

**Conclusion from the above analysis**

1. The sales are highest in december and lowest in january.
                      * This time comes in festive or vaccation season in united states.
2. The sales are highest at 9 to 12 and 16 to 19 hr of the day.
                      * This can be used to increase the staff capacity to maximize customers attention or in atteacting customers through advertisements.
3. The sales falls low during the mid of the week and starts to rise till tuesday.
                      * Similar employement can be addressed as in point 2. 
4. The product with highest qunatity_ordered are batteries and this need not be of the highest values as its per unit price is cheap.
                      * Even though Macbook Pro Laptop is at bottom half of the quantity_ordered it has the highest sales because of its per unit price.
5. Top products that are sold together are iphone, Lightning Charging Cable and GooglePhone, USB-C Charging Cable.
                      * This can be expected as both are husband and wife as one cannot leave without the other.                   
